In [130]:
#the data i'll extract from fred uses economic data for each month since 1980
#we will create the dataset now
import pandas_datareader
import fredapi as fa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt          
import datetime as dt                     
import pandas_datareader.data as web   
import requests
import sklearn
api_key = '0b966e54acf151ff1f562b72a4598cc1'
import os

os.environ['FRED_API_KEY'] = api_key
import time
  
begin = time.time()
 
codes = ['BOGZ1FL073161113Q','BOGZ1FL075035503Q','BOGZ1FL072052006Q','BOGZ1FL073164003Q','BOGZ1FL073161133Q','IRLTLT01USQ156N','IR3TCD01USQ156N','JHDUSRGDPBR']
data = web.DataReader(codes, 'fred', start='1972-01-01',end='2022-01-01')
print(data)

#we will use % change between data points for all data points besides the recession indicator
#this is due to the nature of economic variables
#randomly drawn test set of 25% of data, to make sure recessions are included


            BOGZ1FL073161113Q  BOGZ1FL075035503Q  BOGZ1FL072052006Q  \
DATE                                                                  
1972-01-01             6032.0            35518.0               3.83   
1972-04-01             6141.0            35960.0               4.46   
1972-07-01             6287.0            36454.0               4.87   
1972-10-01             6373.0            37162.0               5.33   
1973-01-01             6601.0            37706.0               7.09   
...                       ...                ...                ...   
2021-01-01             1336.0           310321.0               0.07   
2021-04-01             1591.0           320051.0               0.08   
2021-07-01             1325.0           336216.0               0.08   
2021-10-01             1533.0           351607.0               0.08   
2022-01-01             1951.0           353745.0               0.20   

            BOGZ1FL073164003Q  BOGZ1FL073161133Q  IRLTLT01USQ156N  \
DATE   

In [131]:
#this section is for data cleaning before we apply the model
#each input set has the last 3 quarters % change for each variable

from sklearn.model_selection import train_test_split

datay = list(data['JHDUSRGDPBR'])[4:]
data2 = data.pct_change()
datax = data2.to_numpy()[:,:7]

#creating the 3rd quarter ago, 2nd quarter ago, and last quarter, for each data point
q1 = datax[1:-3,:]
q2 = datax[2:-2,:]
q3 = datax[3:-1,:]
comb = np.concatenate((q1,q2,q3),axis=-1)
#combining these previous measurements into one dataset
print(q3.shape)
print(comb.shape)
print(len(datay))


(197, 7)
(197, 21)
197


In [132]:
X_train, X_test, Y_train, Y_test = train_test_split(comb, datay, train_size=0.70, test_size=0.30)
#i chose a 70/30 train/test split to assure the presence of recessions in each test set

from sklearn.ensemble import AdaBoostClassifier

#i boosted the model using ada
ensemblesize = 3000
for b in range(ensemblesize):
    ada_boosting_classif = AdaBoostClassifier()
    ada_boosting_classif.fit(X_train, Y_train)

    Y_preds = ada_boosting_classif.predict(X_test)
    if b == 0:
        Y_predst = Y_preds
    else:
        Y_predst += Y_preds
Y_predst = Y_predst / ensemblesize


print('Test Accuracy : %.3f'%ada_boosting_classif.score(X_test, Y_test))
print('Training Accuracy : %.3f'%ada_boosting_classif.score(X_train, Y_train))

Y_preds2 = np.zeros(Y_predst.shape)
for a in range(len(Y_preds)):
    if Y_predst[a] >= .4:
        Y_preds2[a] = 1
    #looking at type 1 errors, type 2 errors
type1comp = 0
type2comp = 0
type1 = 0
type2 = 0
for a in range(len(Y_preds2)):
    if Y_preds2[a] == 1:
        type1comp += 1
        if Y_test[a] != 1:
            type1 += 1
    else:
        type2comp += 1
        if Y_test[a] == 1:
            type2 += 1
print("Type 1 error rate:"+str(type1/type1comp))
print("Type 2 error rate:"+str(type2/type2comp))
print("Type 1 errors:"+str(type1))
print("Type 2 errors:"+str(type2))


Test Accuracy : 0.867
Training Accuracy : 1.000
Type 1 error rate:0.5
Type 2 error rate:0.041666666666666664
Type 1 errors:6
Type 2 errors:2


In [133]:
from sklearn.ensemble import AdaBoostRegressor

#this section repeats the steps of the above code, but with regression rather than classification

ensemblesize = 3000
for b in range(ensemblesize):
    ada_boost_regressor = AdaBoostRegressor()
    ada_boost_regressor.fit(X_train, Y_train)

    Y_preds = ada_boost_regressor.predict(X_test)
    if b == 0:
        Y_predst = Y_preds
    else:
        Y_predst += Y_preds
Y_predst = Y_predst / ensemblesize
avger = np.sum(abs(np.subtract(Y_predst,Y_test)))/len(Y_predst)
Y_preds2 = np.zeros(Y_predst.shape)
for a in range(len(Y_preds)):
    if Y_predst[a] >= .4:
        Y_preds2[a] = 1

print('Test Accuracy : %.3f'%(Y_preds2 == Y_test).mean())
print("Average Error is " + str(avger))


type1comp = 0
type2comp = 0
type1 = 0
type2 = 0
for a in range(len(Y_preds2)):
    if Y_preds2[a] == 1:
        type1comp += 1
        if Y_test[a] != 1:
            type1 += 1
    else:
        type2comp += 1
        if Y_test[a] == 1:
            type2 += 1
print("Type 1 error rate:"+str(type1/type1comp))
print("Type 2 error rate:"+str(type2/type2comp))
print("Type 1 errors:"+str(type1))
print("Type 2 errors:"+str(type2))


Test Accuracy : 0.933
Average Error is 0.24105571094517536
Type 1 error rate:0.3
Type 2 error rate:0.02
Type 1 errors:3
Type 2 errors:1
